In [1]:
#Imports
import numpy as np

In [2]:

#Data contains all of statistical information from the game
recruit_list = []

for year in range(2002,2018):

    file_ref = "./Recruiting/{}.txt".format(year)
    by_year = np.loadtxt(file_ref,dtype='str',delimiter='\t\n')
    
    print(len(by_year))
    
    if np.mod(len(by_year),9) == 0:
        shaped = by_year.reshape(int(len(by_year)/9),9)
    else:
        shaped = by_year.reshape(int(len(by_year)/10),10)
        shaped = np.delete(shaped,3,1)
                                 
    recruit_list.append(shaped)  
    
print((recruit_list[0])[:3])

891
900
1000
1000
900
1000
1000
1000
1000
1000
900
900
1000
1000
1000
1000
[['1\t' ' \t' 'Texas' '28\t' '6\t' '15\t' '5\t' '3.89\t' '2792']
 ['2\t' ' \t' 'Tennessee' '25\t' '5\t' '16\t' '2\t' '3.96\t' '2534']
 ['3\t' ' \t' 'Georgia' '30\t' '1\t' '18\t' '8\t' '3.5\t' '2447']]


In [3]:
#Let's read in our team map
teams = np.loadtxt("./teams.csv",dtype='str',delimiter=', ',ndmin=2)

def findteam(team_name):
    #Looks for index given a team name
    
    for i in range(len(teams)):
        for j in range(3):
            if teams[i,j] == team_name:
                return i
    #200 is false
    return 200

In [4]:
num_years = len(recruit_list)
num_teams = len(teams)
#We're adding in two years (2000 and 2001) of repeated data from 2002
per_team = np.zeros((num_teams,num_years+2,6))


for year in range(num_years):
    
    recruit_class = recruit_list[year]
    
    for rank in recruit_class:
    
        curr_team = findteam(rank[2])

        if curr_team != 200:
            
            vec = np.zeros(6)
            for i in range(5):
                vec[i] = float(rank[i+3].replace("\t", ""))
            vec[5] = float(rank[8])
            
            if year == 0:
                per_team[curr_team,(0,1,2)] = vec
            else:
                per_team[curr_team,year+2] = vec
#Normalize
curr_means = np.zeros(6)
curr_stds = np.zeros(6)

for i in range(6):
    flat_recruits = np.ndarray.flatten(per_team[:,2:16,i])
    curr_means[i] = np.mean(flat_recruits)
    curr_stds[i] = np.std(flat_recruits)
    per_team[:,:,i] = (per_team[:,:,i] - curr_means[i])/curr_stds[i]

print(curr_means)
print(curr_stds)

[1.95385588e+01 2.74968394e-01 2.81226296e+00 8.27307206e+00
 2.28807206e+00 8.34805942e+02]
[  9.42309171   0.76150624   4.02294636   6.04293702   1.10324565
 740.57039188]


In [5]:
per_team[0:3,:,4]

array([[-2.07394615, -2.07394615, -2.07394615, -2.07394615, -2.07394615,
        -2.07394615, -2.07394615, -2.07394615, -2.07394615, -2.07394615,
        -2.07394615, -0.11608662, -2.07394615, -0.53303819, -2.07394615,
        -2.07394615, -0.27924158, -2.07394615],
       [-0.13421495, -0.13421495, -0.13421495, -0.21579243, -2.07394615,
        -0.0707658 , -0.10702246, -0.00731665, -0.17953577,  0.03800417,
        -0.04357331, -2.07394615, -2.07394615, -2.07394615, -2.07394615,
        -2.07394615, -2.07394615, -2.07394615],
       [ 0.83565065,  0.83565065,  0.83565065,  0.64530319,  0.55466155,
         0.67249569,  1.19821723,  0.8990998 ,  1.29792303,  1.37950051,
         1.20728139,  1.4338855 ,  1.34324386,  1.40669301,  1.47920632,
         1.44294967,  1.39762884,  1.64236128]])

In [6]:
np.save("./Stored NPs/recruiting.npy",per_team)
np.save("./Stored NPs/recruiting_means.npy",curr_means)
np.save("./Stored NPs/recruiting_stds.npy",curr_stds)